# Сборный проект 1

Из открытых источников доступны исторические данные о продажах игр, оценки пользователей и экспертов, жанры и платформы (например, Xbox или PlayStation). Вам нужно выявить определяющие успешность игры закономерности. Это позволит сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.

# Знакомство с данными

In [1]:
import pandas as pd
try:
    df = pd.read_csv('datasets\games.csv')
except:
    df = pd.read_csv(r'datasets\games.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'datasets\\games.csv'

In [ ]:
df.head(10)

In [ ]:
df.info()

### Описание данных

Name — название игры  
Platform — платформа  
Year_of_Release — год выпуска  
Genre — жанр игры  
NA_sales — продажи в Северной Америке (миллионы проданных копий)  
EU_sales — продажи в Европе (миллионы проданных копий)  
JP_sales — продажи в Японии (миллионы проданных копий)  
Other_sales — продажи в других странах (миллионы проданных копий)  
Critic_Score — оценка критиков (максимум 100)  
User_Score — оценка пользователей (максимум 10)  
Rating — рейтинг от организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 </b>
    
Хорошо, что ты добавляешь описание данных - это упрощает понимание проекта для тех, кто с ним ознакамливается.</div>

## 1. Подготовка данных

<div class="alert-danger"> 
<b>Комментарий ревьюера ❌</b> 
    
Допиши в этом шаге, пожалуйста, некоторые объяснения возможных причин пропусков в анализируемых тобой строках. Это часть задания.
    
Также, проверь наличие дубликатов.

### 1.1 Переименование столбцов

In [ ]:
df.columns = df.columns.str.lower()

<div class="alert-warning"> 
<b>Комментарий ревьюера 💡</b> 
   
Добавлю, что можно сделать это и без for: data.columns = data.columns.str.lower() </div>

###  1.2 Приведение типов

year_of_release - к типу datetime, чтобы можно было работать с данными, как со временем   
user_score - к типу float, для выполнения арифмитических операций в дальнейшем 

In [ ]:
df['year_of_release'] = pd.to_datetime(df['year_of_release'],format='%Y')
df.loc[df['user_score'] == 'tbd', 'user_score'] = float('nan') # tbd - на Nan
df['user_score'] = df['user_score'].astype(float) 
df.info()

###  1.3 Замена дубликатов

In [ ]:
df.duplicated().sum()

Явных дубликатов в выборке не оказалось.

In [ ]:
len(df['name'].str.lower().unique())
print(df.groupby('name').count()['platform'].sum())

По имени дубликатов не оказалось также. Что если есть совпадения по платформе и имени?

In [ ]:
df.duplicated(subset=['platform','name']).sum()
df[df.duplicated(subset=['platform','name'],keep=False)].sort_values(by=['platform','name'])

In [ ]:
df = df.drop(index=[659,14244,16230,4127],axis=0)
df.reset_index(drop=True,inplace=True)

Действительно такие есть, но лишь два объекта под индексом 659,14244,16230,4127 - которые является не информативным из пары строк, поэтому его и удалим. Что касательно  Need for Speed: Most Wanted, то в таблице находятся переиздания на платформе за 2012 и 2005 год. 

In [ ]:
df[df.duplicated(subset=['platform','name'],keep=False)].sort_values(by=['platform','name'])

По другим полям уже не может существовать неявных дубликатов, поэтому перейдем к обрабокте пропусков.

### 1.4 Обработка пропусков

In [ ]:
df.isna().sum()

In [ ]:
names_isna = ['name','year_of_release','genre']
for name in names_isna:
    index=df[df[name].isna()].index
    df = df.drop(index).reset_index(drop=True)

Пропуски, которые сложно восстановить
* **critic_score**,**user_score** - много пропусков одновременно нет, так как мб не было площадок, где люди могли бы оставлять отзывы об играх, либо же эти самые игры не были размещены на каком-нибудь форуме - в зависимости от года выпуска игры
* **rating** - пропуски могут быть из-за того, что возрастной рейтинг ставился в другой системе, которая не соотносится с ESRB, либо системы оценки игры раньше до какого-то года не могло быть, поэтому им не был присужден рейтинг  
Сами по себе пропуски не восстановимы из данных, что имеются, поэтому оставим их, держа в голове, что там они есть.

In [ ]:
df['rating'].fillna('ND',inplace=True) # ND - not defiened - не определен
df.isna().sum()

ND вместо Nan - как заглушка на категориальные данные

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 </b>

Хорошо, что ты заменяешь эти пропуски, это поможет в конце при построении "потрета пользователя" в зависимости от региона.

### 1.5 Обработка данных

In [ ]:
df['total_sales'] = df[['na_sales','eu_sales','jp_sales', 'other_sales']].sum(axis = 1)
df.head()

<div class="alert-warning"> 
<b>Комментарий ревьюера 💡</b> 

Также, можно использовать конструкцию, не переписывая 'df' множнство раз:
    
    df[['na_sales','eu_sales','jp_sales', 'other_sales']].sum(axis = 1),
    
где указание на ось axis = 1 означает суммирование по строкам.

## Исследовательский анализ данных

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

**Важны ли данные за все периоды?**

In [ ]:
df['year_of_release'].hist(bins=36)

<div class="alert-warning"> 
<b>Комментарий ревьюера 💡</b> 
    
Хорошо, что ты визуализируешь эти данные! В данном случае график с непрерывной линией был бы предпочитетельнее - поскольку речь идет именно о динамике на временном отрезке (но можно оставить и такую диаграмму).

Начиная с **2014 года по 2016 год - актуальный период**, на который стоит ориентироваться, поскольку видно, что тренды быстро меняются, поэтому чтобы не оказаться в заблуждение и не рассматривать "мертвые" тренды, возьмем достаточно короткий период за актуальный.

*Рассмотрим топ-5 платформ по суммарным продажам за все время их существования*

In [ ]:
total_sales_top = df.pivot_table(index='platform',values='total_sales',aggfunc={'sum'}).sort_values(['sum'],ascending=False)[:5]
total_sales_top

**За какой характерный срок появляются новые и исчезают старые платформы?**  
*Построим распределение продаж каждой платформы по годам*

In [ ]:
for platform in total_sales_top.index:
    df_info = df.query('platform == @platform').pivot_table(index='year_of_release',values='total_sales',aggfunc={'sum'})
    years= [el.year for el in df_info.index.date] # преобразуем индексы в года
    sns.barplot(x=years,y=df_info['sum']).set(title=platform,ylabel='total_sum')
    plt.show()

*Характерный срок жизни для игровой платформы приблизительно **9-10 лет**, также на примере серии PlayStation - можно увидеть, что новые платформы появляются через **6 лет** после выхода предыдущей.*

**Отфильтруем данные для работы, так, чтобы игры, соотвествовали актуальному периоду** 

Период жизни платформ около 10 лет, что достаточно динамично для рынка, поэтому для прогноза на предстоящий год, возьмем данные за последние 3 года.

In [ ]:
df_actually = df.query('year_of_release >= 2014') # таблица, соответств. актуальному периоду

<div class="alert-danger"> 
<b>Комментарий ревьюера ❌</b> 
    
Для целей прогнозирования продаж на следующий год даже в традиционных моделях бизнеса редко берут данные более чем за 2-3 года. А в такой динамично меняющейся индустрии, как компьютерные игры и вовсе не стоит брать слишком большой временной интервал - иначе обязательно захватишь уже отжившие тренды, которые зависят не только от выхода новой игровой платформы (просто резкий спад продаж может произойти и за достаточно короткий срок).

В данном случае идеальным периодом будет считаться 2014-2016 гг./2015-2016 гг - т.е. период в 2-3 года с сохранением данных за 2016 г.

Поменяй это, пожалуйста.</div>

**Какие платформы лидируют по продажам, растут или падают?**

In [ ]:
df_actually_groped_pf = (df_actually
     .pivot_table(index=['platform','year_of_release'],
                        values='total_sales',
                        aggfunc='sum')
     .reset_index() # избавляемся от мульти-индекса
    )

In [ ]:
for name_platform in df_actually['platform'].unique():
    pf_years = df_actually_groped_pf.query('platform == @name_platform')
    years= [el.year for el in pf_years['year_of_release']]
    sns.barplot(y=pf_years['total_sales'],x=years).set(title=name_platform,ylabel='total_sum')
    plt.show()

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 </b>

Хорошо, что ты пользуешься здесь методом циклов.

**Нам интересны данные, как минимум, об игровых платформах, у которых есть продажи за 2016 год. Среди таких:**
- [ ] Wii  
 *Имеет убывающий тренд, и находится на минимуме своей популярности для разработчиков игр/ Должно быть причина в том, что 
 в 2012 году появляется WiiU, которая является продолжением линейки, к тому же на 2017 год - платформе будет 11 лет, что выбивается из продолжительности жизни платформ*
- [ ] X360  
 *Аналогичная ситуация как у Wii, вышло следующее поколение платформы - Xbox One*
- [ ] PS3  
 *Все точно также как у Wii и X360, комьюнити отдает предпочтение PS4*
- [x] PS4  
 *Платформа достаточно молодая, ей всего 4 года, но уже наблюдается просадка в 2016 году, возможно, что-то произошло с индустрией, либо это связано с тем, что в данных неполный 2017 год*
- [ ] 3DS  
 *Во-первых, есть тренд - убывающий, то есть количество проданных копий игр на этой платформе становится меньше. Во-вторых, по масштабу может показаться, что общий объем денег достаточно высокий на 2016 год, но на самом деле он сопоставим с минимумом для остальных вышеперечисленных исключенных из списка платформ*
- [x] PC  
 *Продолжительность платформы в данном случае не измеримо, так как компьютеры остаются актуальными всегда. Если рассмотреть распределение, то кажется, что продажи игр на PC имеют периодичность, и может быть в 2017 году окажется рост продаж.*
- [x] XOne  
 *Аналогично как у PS4, в обоих случаях должен быть рост*
- [x] WiiU  
 *Наблюдается некоторый отскок проданных копий вниз, но, возможно, это из-за недостатка данных за 2016 год, в таком случае объем копий - достаточный, чтобы назваться популярной платформой на актуальный период*
- [ ] PSV  
 *Продажи явным образом не растут. Либо остаются на том же уровне, либо уменьшаются - во всяком случае по своему объему, они дотстаточно маленькие*  
 ______________________________________________________________________________________________________
 *PSP* - платформа, на которой, насколько нам известно из данных, не выпускались игры в 2016 году, и возможно она уже не интересна нам  
 **Вывод: PS4, XOne, PC, WiiU - самые перспективные платформы на 2017 год** 

**Рассмотрим диаграмму размаха продаж для платформ**

In [ ]:
plt.figure(figsize=(17,8))
sns.boxplot(x='platform',y='total_sales',data=df_actually).set(title='Диаграмма размаха глобальных продаж',ylabel='Продажи')
plt.ylim(0,3);

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 </b>

Отлично, что ты используешь библиотеку seaborn, а не стандартные инструменты matplotlob. Так "ящики с усами" получаются намного более репрезентативными.

PC,PSV, DC, PSP - имееют самую маленькую медиану по продажам среди всех платформ  
WiiU, XOne и PS4 - имеют одинаковую медиану, значением около 0,25*10^6 проданных копий  
PS4 и X360 - верхняя граница этих платформ, наибольшая в акутальном периоде, за ними по уровню границы будут - XOne и PS3

**Влияют ли на продажи отзывы пользователей и критиков?**  
Проверим данную гипотезу на примере популярных платформ, которые мы оценивали ранее

In [ ]:
def corr_info(name):
    df_corr = df_actually.query('platform == @name')
    critic_corr = df_corr.plot(kind='scatter',x='critic_score',y='total_sales',alpha=0.25,title=name);
    user_corr = df_corr.plot(kind='scatter',x='user_score',y='total_sales',alpha=0.25);
    print(f'''
        Коэффициент корреляции продаж от оценок пользователей - {df_corr['total_sales'].corr(df_corr['user_score'])} для {name}
        Коэффициент корреляции продаж от оценок критиков - {df_corr['total_sales'].corr(df_corr['critic_score'])} для {name}
    ''',end='\n------------------------------------------------------------------------------------------------\n')
    
    

In [ ]:
for name in ['PC','XOne','PS4','WiiU']:
    x = corr_info(name)

* Между оценками пользователей и количеством проданных копий корреляция не существует, кроме как у платформы WiiU, в случае которой можно предположить, что игроки WiiU относятся внимательно к оценкам других игроков
* А вот в случае оценков критиков есть некоторая положительная связь, что может обозначать, что мнение критиков слабым образом, но влияет на количество проданных копий среди всех платформ

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 </b>

Согласен, результаты везде соответствуют примерно одному выводу. Зависимость продаж от оценки критиков слабая. От оценки игроков - практически не наблюдается.

**Какие жанры самые прибыльные? Что об этих жанрах можно сказать? Выделяются ли жанры с высокими и низкими продажами?**

In [ ]:
df_actually_genre = df_actually.pivot_table(index='genre',values='total_sales',aggfunc=('sum','mean','median','count')).sort_values(by='sum',ascending=False)

In [ ]:
df_actually_genre

* **Shooter** - самый прибыльный жанр, так как у игр в этом жанре медианное и среднее значение проданных миллион копий - наибольшее
* **Sports** и **Action** следующие жанры, разделяющие 2 и 3 место по этим критеряим, соотвественно 
* **Adventure** - достаточно популярный жанр по числу игр, но имеющий низкие показатели по продажам
* **Puzzle и Strategy** самые не популярные и плохо продающиеся жанры игр

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 </b>

Хорошо, что ты смотришь именно на медианные значения! Это верный подход при поиске наиболее прибыльного жанра.

# Портрет пользователя каждого региона

### Топ 5 платформ по регионам

In [ ]:
regions = ['na_sales','eu_sales','jp_sales']
df_region = df_actually.pivot_table(index='platform',values=regions,aggfunc=('sum'))
fig=plt.figure(figsize=(15,12))
fig.suptitle('Доли продаж платформ по регионам',fontsize=18)
i=1;
for region in (regions):
    ax=fig.add_subplot(1,3,i)
    if(region==regions[0]):
        name='Северная Америка'
    elif(region==regions[1]):
        name='Европа'
    else: 
        name='Япония'
    df_region[region].plot(kind='pie', autopct='%.0f%%',figsize=(19,7),title=name,ax=ax,ylabel='')
    df_region['%_total_sales_'+region] = round(df_region[region]/df_region[region].sum()*100,0)
    i+=1

<div class="alert-warning"> 
<b>Комментарий ревьюера 💡</b> 
    
При составлении портрета пользователей лучше будет построить диаграмы рядом с помощью subplots (не забывая при этом о "двухуровневых заголовках" - и у всех трех графиков вместе, и у каждого из трех по отдельности).</div>

In [ ]:
region_top = pd.DataFrame(
    {region : df_region.sort_values(by=region,ascending=False).index[:5].array for region in regions}
)
region_percentage = pd.DataFrame(
    {region : df_region.sort_values(by=region,ascending=False)['%_total_sales_'+region][:5] for region in regions}
)
region_top.index = [i+1 for i in range(5)] # индекс имееет значение в топе для региона
region_percentage.fillna(0,inplace=True)
region_percentage = region_percentage.T
region_percentage

In [ ]:
region_top = region_top .T
region_top

In [ ]:
region_info = region_top.join(region_percentage)
region_info

**Из конечной таблицы очевидно, что по регионам в период 2007-2016 год, самыми популярными платформами были:**
* В Северной Америке - **PS4**
* В Европе - **PS4**
* В Японии - **3DS**  
_______________________________
Также стоит пояснить, что нули существующие в таблице - не абсолютные, то есть на самом деле у этих платформ есть какая-то доля в этом регионе, но 0 обозначает, что платформа не входит в топ 5 продажам для этого региона.  
* Поэтому инетерсным образом можно заключить, что PSP и 3DS платформы являются особенными для японского рынка, что являются единственными платформами входящими в  топ 5, какого либо региона. 
* Помимо этого в Японии мало общего в любимых платформах для Европы и Сев. Америки - PS4 и X360 также не входит в их топ покупок.  
* В Северной Америке и Европе выбор платформ остается одинаковым, но ранжируется их порядок в этом самом топе, то есть между их платформами в топе - нет различий.

### Топ  - 5 жанров игр по регионам.

In [ ]:
df_region_genre = df_actually.pivot_table(index='genre',values=regions,aggfunc=('sum'))
fig=plt.figure(figsize=(18,16))
fig.suptitle('Доли жанров игр по регионам',fontsize=18)
i=1;
for region in regions:
    ax=fig.add_subplot(1,3,i)
    if(region==regions[0]):
        name='Северная Америка'
    elif(region==regions[1]):
        name='Европа'
    else: 
        name='Япония'
    df_region_genre[region].plot(kind='pie', autopct='%.0f%%',figsize=(20,8),title=name,ax=ax,ylabel='')
    i+=1

In [ ]:
region_top_genre = pd.DataFrame(
    {region : df_region_genre.sort_values(by=region,ascending=False).index[:5].array for region in regions}
)
region_top_genre.index = [i+1 for i in range(5)]
region_top_genre

В топе популярных жанров по регионам разницы между Северной Америкой и Европой не наблюдается, даже если посмотреть Role-Playing на круговой диаграмме для Европы то он составляет практически такую же долю, как и Racing, с точностью до целых процентов. 
В Японии кардинально другие предпочтения в жанрах.

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 </b>

Приавльный промежуточный вывод. Япониия в этом смысле действительно отличается.

**Влияет ли рейтинг ESRB на продажи в отдельном регионе?**

In [ ]:
for region in regions:
    df_rate = df_actually.pivot_table(index='rating',values=region,aggfunc='sum').plot(kind='line',grid=True,figsize=(6,2))

Сначала рассмотрим Северную Америку и Европу. Можно представить, что по горизонтали с каждой следующей категорией увеличивается возрастной рейтинг, кроме категории ND - которая не описывает, к какой категории относятся проданные копии игр.
В таком случае, чем выше возрастной рейтинг, тем меньшее число копий можно ожидать продать.  
По Японии очень много ND - должно быть это связано с **CERO** (Computer Entertainment Rating Organization), которая устанавливает возрастной рейтинг в своих категориях. А данные, что нам известны, это игры, созданные и выпущенные сначала на не японской территории. Но даже без этого факта в Японии наблюдается такая же зависимость, как и в Америке с Европой.

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 </b>

Верный вывод! Тут пригодилась замена, сделанная в самом начале. Можно сказать, что рейтинг ESRB не влияет на продажи в Японии, поскольку большинство продаж здесь приходится на "ND".

# Проверка гипотез

**1. Средние пользовательские рейтинги платформ Xbox One и PC одинаковые.**
* Нулевая гипотеза: Средний пользовательский рейтинг Xbox One совпадает со среднем пользовательским рейтингом PC
* Альтернативная гипотеза: Средний пользовательский рейтинг *НЕ* совпадает со среднем пользовательским рейтингом PC
______________________
Поскольку гипотеза сформулирована о проверке среднего 2 генеральных совокупностей, то мы будем использовать критерий Стьюдента или так называемый t-тест.  
Примем критический уровень статистической значимости - 0.05.

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 </b>

Правильное обоснование использования критерия Стьюдента.

In [ ]:
from scipy import stats as st
import numpy as np
alpha=0.05
df_xone = df_actually[df_actually['user_score'].notna()].query('platform == "XOne"')['user_score']#ОЧИСТИМ ОТ NAN, ТАК КАК Т-ТЕСТ ЧУВСТВИТЕЛЕН 
df_pc = df_actually[df_actually['user_score'].notna()].query('platform == "PC"')['user_score'] #КПРОПУСКОМ
results = st.ttest_ind(df_xone,
                       df_pc,
                      equal_var=False) # у выборок должна быть разная дисперсия
print('p-value:',results.pvalue)
if (results.pvalue < alpha):
    print('Отвергаем нулевую гипотезу')
else:
    print('Нулевую гипотезу не получилось отвергнуть')



**Вывод:** средние пользовательские оценки у Xbox One и PC - разные.

2. **Средние пользовательские рейтинги жанров Action (англ. «действие», экшен-игры) и Sports (англ. «спортивные соревнования») разные.**
* Нулевая гипотеза: Средние пользовательские рейтинги жанров Action и Sports - равны 
* Альтернативная гипотеза: Средние пользовательские рейтинги жанров Action и Sports - разные
______________________
Поскольку гипотеза сформулирована о проверке среднего 2 генеральных совокупностей, то мы будем использовать критерий Стьюдента или так называемый t-тест.  
Примем критический уровень статистической значимости - 0.05.

In [ ]:
alpha=0.05
df_action = df_actually[df_actually['user_score'].notna()].query('genre == "Action"')['user_score']#ОЧИСТИМ ОТ NAN, ТАК КАК Т-ТЕСТ ЧУВСТВИТЕЛЕН 
df_sports = df_actually[df_actually['user_score'].notna()].query('genre == "Sports"')['user_score'] #КПРОПУСКОМ
results = st.ttest_ind(df_action,
                       df_sports,
                      equal_var=False) # у выборок должна быть разная дисперсия
print('p-value:',results.pvalue)
if (results.pvalue < alpha):
    print('Отвергаем нулевую гипотезу')
else:
    print('Нулевую гипотезу не получилось отвергнуть')
    

**Вывод:** Да, действительно, средние пользовательских рейтинги этих жанров - разные.

# Общий вывод
1. В ходе работы были предобработаны данные - удалены пропуски, где это возможно, приведены типы данных, установлены заглушки в категориальных данных.
2. Средняя жизнь платформы оказалось 9-10 лет.
3. За актуальный период в данных был выбран период с 2014 по 2016 год.
4.  PS4, XOne, PC, WiiU - самые перспективные платформы на 2017 год, по потенциальному объекму проданных копий
5.  PS4 и XOne - платформы среди акутального периода, у которых самая высокая верхняя граница и медиана по числу проданных копий игр для какой либо игры
6. Shooter - самый прибыльный жанр игр
7. Пользователи Севереной Америки и Европы не отличаются в выборе платформы и жанров, в отличии от Японии, где абсолютно своя культура игр
8. Чем меньше возрастной ценз, тем больше проданных копий окажется у игры

<div class="alert-success"> 
<b>Комментарий ревьюера 👍 </b>
    
Объемный, качественно написанный вывод! Всё отражено верно.

<div class="alert-success"> 
<b>Общий комментарий ревьюера </b> 
    
Отличный проект, тобою проделана большая работа, ты молодец!

Хвалю за наличие хороших промежуточных (и общего) выводов, комментариев и рассуждений - твой проект интересно проверять. Также очень хорошее оформление проекта - в этом плане сделано все именно так, как требуется. По коду вопросов в целом нет.

Что обязательно стоит изменить: 
    
1. Изменить актуальный период
2. Проверить дубликаты, описать причины пропусков.

Также, я оставил некоторое количество желтых комментариев. Думаю, они будут полезны и ты также что-либо поменяешь, потому что это сделает твой проект лучше.

Буду ждать доработок :)
    
Удачи!

<div class="alert alert-info"> <b>Комментарий студента:
    </b> Спасибо за предложенные варианты с кодом. Круговые диагрммы также исправил. Проверил дубликаты и описал причины с пропусками. Спасибо за работу!</div>
    


<div class="alert-success"> 
<b>Комментарий ревьюера 👍 Вторая итерация</b>

Все, что я отмечал, исправлено. У тебя получился качественно сделанный проект, а теперь он стал ещё лучше. Принимаю! 
    
Успехов в дальнейшей учебе! 